In [5]:
import os
import time
import json
from glob import glob
from dotenv import load_dotenv
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
from langchain_upstage import UpstageEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import tiktoken

STATE_FILE = "upload_progress.json"

# 환경 세팅
load_dotenv()
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
pc = Pinecone()
index_name = 'insurance-index-upstage'

# tiktoken 세팅 (Upstage solar-embedding-1과 호환)
tokenizer = tiktoken.get_encoding("cl100k_base")
MAX_TOKEN = 4000
CHUNK_OVERLAP = 500   # 토큰 기준

docx_files = glob('./docxfile/*.docx')
all_docs = []

print(f"총 {len(docx_files)}개 파일에서 chunk 생성 시작...")

def split_by_token(text, max_token=4000, overlap=500):
    tokens = tokenizer.encode(text)
    total = len(tokens)
    chunks = []
    start = 0
    while start < total:
        end = min(start + max_token, total)
        chunk_tokens = tokens[start:end]
        # 만약 실제 chunk가 max_token 초과면 자름 (방어적)
        while len(chunk_tokens) > max_token:
            chunk_tokens = chunk_tokens[:max_token]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
        start += max_token - overlap  # overlap 만큼 겹치기
    return chunks

for file_idx, filepath in enumerate(docx_files, 1):
    filename = os.path.basename(filepath)
    company_name = os.path.splitext(filename)[0]
    print(f"[{file_idx}/{len(docx_files)}] {filename} → 회사명: {company_name}")

    loader = Docx2txtLoader(filepath)
    docs = loader.load()
    text = docs[0].page_content

    # 토큰 기준으로 안전하게 분할
    chunks = split_by_token(text)
    print(f"    - {len(chunks)}개의 chunk 생성 (토큰 기준)")

    doc_objs = [Document(page_content=chunk, metadata={"company": company_name}) for chunk in chunks]
    all_docs.extend(doc_objs)

print(f"\n총 {len(all_docs)}개의 Document 객체 생성 완료.")
for idx, doc in enumerate(all_docs):
    tokens = tokenizer.encode(doc.page_content)
    if len(tokens) > 3500:
        print(f"WARNING: {idx}번 청크가 {len(tokens)}토큰 (최대 4000 초과), 자동 잘림!")
        doc.page_content = tokenizer.decode(tokens[:4000])
print("임베딩 및 Pinecone 업로드 시작...")

batch_size = 20
total_batches = (len(all_docs) + batch_size - 1) // batch_size

# 상태 파일 로딩
if os.path.exists(STATE_FILE):
    with open(STATE_FILE, "r", encoding="utf-8") as f:
        state = json.load(f)
    uploaded_batches = set(state.get("uploaded_batches", []))
    print(f"\n이전 진행 내역 발견: {len(uploaded_batches)}개 배치 이미 업로드됨. 이어서 진행합니다.")
else:
    uploaded_batches = set()

success_count = len(uploaded_batches) * batch_size

for batch_idx, i in enumerate(range(0, len(all_docs), batch_size), 1):
    batch_num = batch_idx
    batch_docs = all_docs[i:i+batch_size]
    if batch_num in uploaded_batches:
        print(f"\n[{batch_num}/{total_batches}] 이미 업로드됨 → 건너뜀")
        continue

    print(f"\n[{batch_num}/{total_batches}] {i+1}~{i+len(batch_docs)}번째 문서 임베딩+업로드 중...")

    retry = 0
    while True:
        try:
            PineconeVectorStore.from_documents(
                documents=batch_docs,
                embedding=embeddings,
                index_name=index_name,
            )
            success_count += len(batch_docs)
            print(f"    ✔️ {len(batch_docs)}개 업로드 성공 (누적 {success_count}/{len(all_docs)})")
            uploaded_batches.add(batch_num)
            with open(STATE_FILE, "w", encoding="utf-8") as f:
                json.dump({"uploaded_batches": list(uploaded_batches)}, f)
            break
        except Exception as e:
            if '429' in str(e) or 'RateLimit' in str(e):
                retry += 1
                wait_time = 60 + retry * 30   # 반복마다 점진적 대기 증가
                print(f"    ❗ RateLimit(429) 발생! {wait_time}초 후 재시도 (시도 {retry})")
                time.sleep(wait_time)
            else:
                print(f"    ❌ 알 수 없는 에러: {str(e)}")
                raise

    # 업로드 성공 후 batch 간 추가 대기 (최소 60초)
    print("    ⏳ 다음 배치까지 20초 대기")
    time.sleep(10)

print(f"\n=== Pinecone 전체 업로드 완료! 총 업로드: {success_count}개 ===")
if os.path.exists(STATE_FILE):
    os.remove(STATE_FILE)
    print(f"진행 상태파일({STATE_FILE}) 삭제 완료")


총 11개 파일에서 chunk 생성 시작...
[1/11] AXA손해보험.docx → 회사명: AXA손해보험
    - 293개의 chunk 생성 (토큰 기준)
[2/11] DB손해보험.docx → 회사명: DB손해보험
    - 478개의 chunk 생성 (토큰 기준)
[3/11] KB자동차보험.docx → 회사명: KB자동차보험
    - 794개의 chunk 생성 (토큰 기준)
[4/11] MG손해보험.docx → 회사명: MG손해보험
    - 69개의 chunk 생성 (토큰 기준)
[5/11] 롯데손해보험.docx → 회사명: 롯데손해보험
    - 80개의 chunk 생성 (토큰 기준)
[6/11] 메리츠.docx → 회사명: 메리츠
    - 69개의 chunk 생성 (토큰 기준)
[7/11] 삼성화재.docx → 회사명: 삼성화재
    - 92개의 chunk 생성 (토큰 기준)
[8/11] 캐롯손해보험.docx → 회사명: 캐롯손해보험
    - 134개의 chunk 생성 (토큰 기준)
[9/11] 하나손해보험.docx → 회사명: 하나손해보험
    - 256개의 chunk 생성 (토큰 기준)
[10/11] 현대해상.docx → 회사명: 현대해상
    - 295개의 chunk 생성 (토큰 기준)
[11/11] 흥국화재.docx → 회사명: 흥국화재
    - 74개의 chunk 생성 (토큰 기준)

총 2634개의 Document 객체 생성 완료.
임베딩 및 Pinecone 업로드 시작...

이전 진행 내역 발견: 29개 배치 이미 업로드됨. 이어서 진행합니다.

[1/132] 이미 업로드됨 → 건너뜀

[2/132] 이미 업로드됨 → 건너뜀

[3/132] 이미 업로드됨 → 건너뜀

[4/132] 이미 업로드됨 → 건너뜀

[5/132] 이미 업로드됨 → 건너뜀

[6/132] 이미 업로드됨 → 건너뜀

[7/132] 이미 업로드됨 → 건너뜀

[8/132] 이미 업로드됨 → 건너뜀

[9/132] 이미 업로드됨 → 건너뜀

[10/13

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4000 tokens, but your request contains 5165 tokens. Please reduce the length of your input text or select only the most relevant portions to include in your request. For information on token counting methods and model-specific limits, please refer to our API reference documentation (https://console.upstage.ai/api/embeddings)", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_body'}}

In [6]:
import os
import time
import json
from glob import glob
from dotenv import load_dotenv
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
from langchain_upstage import UpstageEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import tiktoken

STATE_FILE = "upload_progress.json"

# 환경 세팅
load_dotenv()
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
pc = Pinecone()
index_name = 'insurance-index-upstage'

tokenizer = tiktoken.get_encoding("cl100k_base")
MAX_TOKEN = 4000
CHUNK_OVERLAP = 500

docx_files = glob('./docxfile/*.docx')
all_docs = []

print(f"총 {len(docx_files)}개 파일에서 chunk 생성 시작...")

def split_by_token(text, max_token=MAX_TOKEN, overlap=CHUNK_OVERLAP):
    tokens = tokenizer.encode(text)
    total = len(tokens)
    chunks = []
    start = 0
    while start < total:
        end = min(start + max_token, total)
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens)
        # decode 결과가 max_token을 넘을 수도 있으니 다시 체크
        chunk_tokens = tokenizer.encode(chunk_text)
        if len(chunk_tokens) > max_token:
            chunk_text = tokenizer.decode(chunk_tokens[:max_token])
        chunks.append(chunk_text)
        start += max_token - overlap
    return chunks

# 1. 문서별 청크 생성
for file_idx, filepath in enumerate(docx_files, 1):
    filename = os.path.basename(filepath)
    company_name = os.path.splitext(filename)[0]
    print(f"[{file_idx}/{len(docx_files)}] {filename} → 회사명: {company_name}")

    loader = Docx2txtLoader(filepath)
    docs = loader.load()
    text = docs[0].page_content

    # 토큰 기준으로 청크 생성
    chunks = split_by_token(text)
    print(f"    - {len(chunks)}개의 chunk 생성 (토큰 기준)")

    doc_objs = [Document(page_content=chunk, metadata={"company": company_name}) for chunk in chunks]
    all_docs.extend(doc_objs)

print(f"\n총 {len(all_docs)}개의 Document 객체 생성 완료.")

# 2. 업로드 전 2중 방어: 모든 청크 토큰 길이 체크 및 자르기
for idx, doc in enumerate(all_docs):
    tokens = tokenizer.encode(doc.page_content)
    if len(tokens) > MAX_TOKEN:
        print(f"WARNING: {idx}번 청크가 {len(tokens)}토큰 (최대 {MAX_TOKEN} 초과), 자동 잘림!")
        doc.page_content = tokenizer.decode(tokens[:MAX_TOKEN])

print("임베딩 및 Pinecone 업로드 시작...")

batch_size = 20
total_batches = (len(all_docs) + batch_size - 1) // batch_size

# 상태 파일 로딩
if os.path.exists(STATE_FILE):
    with open(STATE_FILE, "r", encoding="utf-8") as f:
        state = json.load(f)
    uploaded_batches = set(state.get("uploaded_batches", []))
    print(f"\n이전 진행 내역 발견: {len(uploaded_batches)}개 배치 이미 업로드됨. 이어서 진행합니다.")
else:
    uploaded_batches = set()

success_count = len(uploaded_batches) * batch_size

for batch_idx, i in enumerate(range(0, len(all_docs), batch_size), 1):
    batch_num = batch_idx
    batch_docs = all_docs[i:i+batch_size]
    if batch_num in uploaded_batches:
        print(f"\n[{batch_num}/{total_batches}] 이미 업로드됨 → 건너뜀")
        continue

    print(f"\n[{batch_num}/{total_batches}] {i+1}~{i+len(batch_docs)}번째 문서 임베딩+업로드 중...")

    retry = 0
    while True:
        try:
            PineconeVectorStore.from_documents(
                documents=batch_docs,
                embedding=embeddings,
                index_name=index_name,
            )
            success_count += len(batch_docs)
            print(f"    ✔️ {len(batch_docs)}개 업로드 성공 (누적 {success_count}/{len(all_docs)})")
            uploaded_batches.add(batch_num)
            with open(STATE_FILE, "w", encoding="utf-8") as f:
                json.dump({"uploaded_batches": list(uploaded_batches)}, f)
            break
        except Exception as e:
            if '429' in str(e) or 'RateLimit' in str(e):
                retry += 1
                wait_time = 20  # 항상 20초 대기 (고정)
                print(f"    ❗ RateLimit(429) 발생! {wait_time}초 후 재시도 (시도 {retry})")
                time.sleep(wait_time)
            else:
                print(f"    ❌ 알 수 없는 에러: {str(e)}")
                raise

    # 업로드 성공 후 batch 간 20초 대기
    print("    ⏳ 다음 배치까지 20초 대기")
    time.sleep(20)

print(f"\n=== Pinecone 전체 업로드 완료! 총 업로드: {success_count}개 ===")
if os.path.exists(STATE_FILE):
    os.remove(STATE_FILE)
    print(f"진행 상태파일({STATE_FILE}) 삭제 완료")


총 11개 파일에서 chunk 생성 시작...
[1/11] AXA손해보험.docx → 회사명: AXA손해보험
    - 293개의 chunk 생성 (토큰 기준)
[2/11] DB손해보험.docx → 회사명: DB손해보험
    - 478개의 chunk 생성 (토큰 기준)
[3/11] KB자동차보험.docx → 회사명: KB자동차보험
    - 794개의 chunk 생성 (토큰 기준)
[4/11] MG손해보험.docx → 회사명: MG손해보험
    - 69개의 chunk 생성 (토큰 기준)
[5/11] 롯데손해보험.docx → 회사명: 롯데손해보험
    - 80개의 chunk 생성 (토큰 기준)
[6/11] 메리츠.docx → 회사명: 메리츠
    - 69개의 chunk 생성 (토큰 기준)
[7/11] 삼성화재.docx → 회사명: 삼성화재
    - 92개의 chunk 생성 (토큰 기준)
[8/11] 캐롯손해보험.docx → 회사명: 캐롯손해보험
    - 134개의 chunk 생성 (토큰 기준)
[9/11] 하나손해보험.docx → 회사명: 하나손해보험
    - 256개의 chunk 생성 (토큰 기준)
[10/11] 현대해상.docx → 회사명: 현대해상
    - 295개의 chunk 생성 (토큰 기준)
[11/11] 흥국화재.docx → 회사명: 흥국화재
    - 74개의 chunk 생성 (토큰 기준)

총 2634개의 Document 객체 생성 완료.
임베딩 및 Pinecone 업로드 시작...

이전 진행 내역 발견: 29개 배치 이미 업로드됨. 이어서 진행합니다.

[1/132] 이미 업로드됨 → 건너뜀

[2/132] 이미 업로드됨 → 건너뜀

[3/132] 이미 업로드됨 → 건너뜀

[4/132] 이미 업로드됨 → 건너뜀

[5/132] 이미 업로드됨 → 건너뜀

[6/132] 이미 업로드됨 → 건너뜀

[7/132] 이미 업로드됨 → 건너뜀

[8/132] 이미 업로드됨 → 건너뜀

[9/132] 이미 업로드됨 → 건너뜀

[10/13

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4000 tokens, but your request contains 5165 tokens. Please reduce the length of your input text or select only the most relevant portions to include in your request. For information on token counting methods and model-specific limits, please refer to our API reference documentation (https://console.upstage.ai/api/embeddings)", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_body'}}

In [30]:
query = "피의자가 음주운전을 해서 피해를 받으면 보상이 있나요?"
retrieved_docs = database.similarity_search(query,
                                            k=3, # 기본 k는 4이며, 최대 k는 100이다.
                                            )
# ai_message = qa_chain.invoke({'query':query})
# ai_message

In [31]:
prompt = f'''[identity]
- 당신은 최고의 한국 보험 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요
[context]는 다음과 같습니다
{retrieved_docs}
Question: {query}'''

In [32]:
ai_message = llm.invoke(prompt)

In [33]:
print(ai_message.content)

네, 음주운전으로 인한 피해의 경우 보험에서 보상이 가능합니다. 특히, 형사합의 지원금 특별약관과 변호사선임비용 지원 특별약관에 따라, 음주운전으로 인한 사고로 피보험자가 법률상 손해배상책임을 지게 되거나 법적 절차가 진행되는 경우, 보험금이 지급될 수 있습니다. 

구체적으로는, 보험약관에서는 피보험자가 무면허운전 또는 음주운전으로 사고를 낸 경우 보험금이 지급되지 않는 경우가 명시되어 있지만, 만약 사고가 발생하고 법률상 손해배상책임이 발생했으며, 관련 서류(예를 들어, 경찰서 교통사고사실확인원, 형사합의서 등)를 제출할 경우 보험금 지급 대상이 될 수 있습니다.

또한, 만약 사고로 인해 형사합의금 또는 변호사 선임 비용이 발생했다면, 해당 내용으로 보험금을 청구할 수 있으며, 실손으로 보상받거나 지원하는 방식으로 처리됩니다.

요약하자면, 음주운전 사고로 피해를 입은 경우, 사고 사실에 대한 적절한 증빙 서류와 함께 보험사에 청구하면 일정 부분 보험혜택을 받을 수 있습니다.
